In [ ]:
# !pip install gdown --upgrade
# !pip install torchinfo
# !pip install torchviz
# !pip install wandb
# !pip install -U imbalanced-learn
# !pip install scikit-maad

In [ ]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from sklearn import preprocessing
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchinfo import summary
from tqdm.notebook import tqdm
from scipy import signal

torch.__version__ # 1.10.0+cu111

In [ ]:
!wandb login # 33d4ac1d40249bd82711d261c69583461dded7c7

In [ ]:
wandb.login()

In [ ]:
# ! gdown --fuzzy https://drive.google.com/file/d/1iVah8T28-ib5YJUIyjI-sSbD7DiEfkuV/view?usp=share_link

# Preparing data

In [ ]:
# !unzip uwb-pose-prediction.zip

In [ ]:
annot_df = pd.read_csv('annotations.csv')
annot_df

In [ ]:
class_df = pd.read_csv('classes.csv')
class_df

In [ ]:
#Range time data
def range_time(IQ_data, overlap = 1): 
    n_rd_history = 256
    frame = []
    frames = []
    overlap_count = 0
    check = 0
    for iqini in IQ_data:
        if len(frame)<n_rd_history:
            frame.append(iqini)
        else:  
            if check == 0:
                frame1 = np.array(frame)
                frames.append(np.copy(frame1))
                check = 1

            if overlap_count < overlap:
                frame.append(iqini)
                overlap_count = overlap_count+1

            if overlap_count == overlap:
                frame = frame[overlap_count::]
                frame1 = np.array(frame)
                frames.append(np.copy(frame1))
                overlap_count = 0
    return np.stack(frames)

import scipy.fftpack as fft
#Range frequency data
def range_frequency(datas, noise_threshold=None):
    Range_frequency_frame = []
    for data in datas:
        jitter = 1e-10
        # noise_threshold = -45
        dB = True
        #rd_history = np.hanning(n_rd_history)[:, None] * np.array(data)
        # Range-Doppler
        rd = fft.fft(data, axis=0)
        rd = fft.fftshift(rd, axes=0)
        rd = abs(rd)
        if dB:
            rd = 20 * np.log10(rd+jitter)
            if noise_threshold is not None:
                rd[rd < noise_threshold] = noise_threshold
        Range_frequency_frame.append(rd)
    return np.stack(Range_frequency_frame)

def get_time_img(data, data_path = '/train/train/'):
    # output = np.load(f'{data_path}{data_id}')
    output = range_time(data, overlap=256)
    return output.reshape(1,1,-1,56)
    # return output.reshape(-1,56)

def get_freq_img(data, noise_threshold=None):
    # output = np.load(f'{data_path}{data_id}')
    output = range_time(data, overlap=256)
    # output = signal.detrend(output)
    output = range_frequency(output, noise_threshold=noise_threshold)
    return output.reshape(1,1,-1,56)
    # return output.reshape(-1,56)

In [ ]:
# tmp = get_freq_img(np.load('train/train/001b0660-4c6e-4d07-8cd5-cd63578512f7.npy'), noise_threshold=-50)
tmp = np.abs(get_time_img(np.load('train/train/001b0660-4c6e-4d07-8cd5-cd63578512f7.npy')))
print(tmp.shape)
tmp = torch.tensor(tmp.reshape(1,-1,56))
resize = transforms.Resize((256,256))
tmp = resize(tmp)
plt.imshow(tmp.permute(1, 2, 0))

In [ ]:
tmp = np.abs(get_time_img(np.load('train/train/001b0660-4c6e-4d07-8cd5-cd63578512f7.npy')))
print(tmp.shape)
tmp = torch.tensor(tmp.reshape(1,-1,56))
resize = transforms.Resize((2560,2560))
tmp = resize(tmp)
resize = transforms.Resize((256,256))
tmp = resize(tmp)
plt.imshow(tmp.permute(1, 2, 0))

In [ ]:
from pandas.core.common import random_state
def train_test_split_df(df, test_size=0.1):
    test_df = df.sample(frac = test_size, random_state = 42)
    train_df = df.drop(test_df.index)
    return train_df, test_df

In [ ]:
train_df, val_df = train_test_split_df(annot_df, test_size=0.2)
print(train_df.shape, val_df.shape)

In [ ]:
plt.bar(train_df['class'].value_counts().index, height=train_df['class'].value_counts().values)

In [ ]:
plt.bar(val_df['class'].value_counts().index, height=val_df['class'].value_counts().values)

In [ ]:
def read_freq_data(df, main_path='train/train/', noise_threshold=None):
    # features = np.array([], dtype=np.float32).reshape(1,2560,56)
    # labels = np.array([], dtype=np.float32)
    features = np.empty((0,1,2560,56))
    labels = np.array([], dtype=np.float32)
    for i in tqdm(range(len(df))):
        feature_tmp = np.load(f"{main_path}{df.iloc[i]['id']}.npy")
        if feature_tmp.shape != (2560,56):
            continue
        feature_tmp = get_freq_img(feature_tmp, noise_threshold=noise_threshold)
        features = np.concatenate((features, feature_tmp), axis=0)
        labels = np.concatenate((labels, df.iloc[i]['class'].reshape(1)), axis=0)

    return features, labels

In [ ]:
def read_time_data(df, main_path='train/train/'):
    # features = np.array([], dtype=np.float32).reshape(1,2560,56)
    # labels = np.array([], dtype=np.float32)
    features = np.empty((0,1,2560,56))
    labels = np.array([], dtype=np.float32)
    for i in tqdm(range(len(df))):
        feature_tmp = np.load(f"{main_path}{df.iloc[i]['id']}.npy")
        if feature_tmp.shape != (2560,56):
            continue
        feature_tmp = np.abs(get_time_img(feature_tmp))
        features = np.concatenate((features, feature_tmp), axis=0)
        labels = np.concatenate((labels, df.iloc[i]['class'].reshape(1)), axis=0)

    return features, labels

In [ ]:
class MyScaler:
    def __init__(self, x_train, method='min-max'):
        self.method = method
        self.x_train = x_train
        if method == 'min-max':
            self.x_min = x_train.min()
            self.x_max = x_train.max()
        elif method == 'std':
            self.x_mean = x_train.min()
            self.x_std = x_train.std()
        else:
            raise NotImplementedError
    
    def transforms(self, x):
        if self.method == 'min-max':
            return (x - self.x_min)/(self.x_max-self.x_min)
        elif self.method == 'std':
            return (x - self.x_mean)/self.x_std

In [ ]:
from imblearn.over_sampling import SMOTE

noise_threshold = -50

# x_train, y_train = read_freq_data(train_df, noise_threshold=noise_threshold)
# x_val, y_val = read_freq_data(val_df, noise_threshold=noise_threshold)

x_train, y_train = read_time_data(train_df)
x_val, y_val = read_time_data(val_df)

train_size = x_train.shape[0]
sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train.reshape(train_size,-1),y_train)

train_size = x_train.shape[0]
x_train = x_train.reshape(train_size, 1, -1, 56)

# x_scaler = MyScaler(x_train, method='std')
# x_train = x_scaler.transforms(x_train)
# x_val = x_scaler.transforms(x_val)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
class UWBDataset(Dataset):
    def __init__(self, x, y):
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        print(self.x.shape)
        print(self.y.shape)

    def __getitem__(self, index):
        x = self.x[index] # Retrieve feature data
        resize = transforms.Resize((256,256))
        x = resize(torch.tensor(x))
        y = self.y[index] # Retrieve target
        return x, y

    def __len__(self):
        return self.x.shape[0]

In [ ]:
train_dataset = UWBDataset(x_train, y_train)
val_dataset = UWBDataset(x_val, y_val)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True)
for i in train_loader:
    print(i[0].shape)
    plt.imshow(i[0][0].permute(1, 2, 0))
    break

# Create model

In [ ]:
# !pip install timm

In [ ]:
# !pip install torchinfo

In [ ]:
import timm
import os 
from torchinfo import summary
# os.mkdir('/kaggle/working/output')

In [ ]:
def create_model(model_name, pretrained = True, num_classes=7):
    model = timm.create_model(model_name, pretrained = pretrained, num_classes = num_classes)
    return model

In [ ]:
def search_model(name):
    for model_name in timm.list_models():
        if name in model_name:
            print(model_name)
    

In [ ]:
####################################################### List of model #######################################################
name = 'efficientnetv2'
search_model(name)

In [ ]:
model_name = 'efficientnetv2_s'

model = create_model(model_name)
model

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True, num_classes=7)
        self.model.conv_stem = torch.nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

    def forward(self, x):
        x = self.model(x)
        # x = nn.Softmax(dim=1)(x) # Softmax layer added to output
        return x

In [ ]:
model = MyModel()

In [ ]:
summary(model)

In [ ]:
model = model.cuda()

# Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
run_name = f'{model_name}_lr0.01_range_time'
result_path = f'results/{run_name}'
if not os.path.exists(result_path):
    os.mkdir(result_path)

config = {
    'architecture': model_name,
    'lr': 0.01,
    'scheduler_factor': 0.5,
    'scheduler_patience': 3,
    'early_stop_patience' : 10,
    'scheduler_min_lr': 1e-4,
    'epochs': 200,
    'batch_size': 4,
    'model_path': f'results/{run_name}/model.pth.tar',
    'best_model_path': f'results/{run_name}/best_model.pth'
}

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# Model
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    'min', 
    factor=config['scheduler_factor'], 
    patience=config['scheduler_patience'],
    min_lr=config['scheduler_min_lr']
)

In [ ]:
train_losses = []
val_losses = []
learning_rates = []
loss_fn = nn.CrossEntropyLoss()
early_stop_counter = 0
best_val_loss = np.inf

# Start wandb run
wandb.init(
    project=run_name,
    config=config,
)

# Log parameters and gradients
wandb.watch(model, log='all')

for epoch in range(config['epochs']):  # loop over the dataset multiple times
    
    # Training
    train_loss = []
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)

    # Flag model as training. Some layers behave differently in training and
    # inference modes, such as dropout, BN, etc.
    model.train()

    print(f"Training epoch {epoch+1}...")
    print(f"Current LR: {current_lr}")

    for i, (inputs, y_true) in enumerate(tqdm(train_loader)):
        # Transfer data from cpu to gpu
        inputs = inputs.to(device)
        y_true = y_true.to(device)
        y_true = y_true.to(torch.int64)

        # Reset the gradient
        optimizer.zero_grad()

        # Predict
        y_pred = model(inputs)

        # Calculate loss
        loss = loss_fn(y_pred, y_true)

        # Compute gradient
        loss.backward()
        
        # Update parameters
        optimizer.step()

        # Log stuff
        train_loss.append(loss)
        
    avg_train_loss = torch.stack(train_loss).mean().item()
    train_losses.append(avg_train_loss)

    print(f"Epoch {epoch+1} train loss: {avg_train_loss:.4f}")
    
    # Validation
    model.eval()
    with torch.no_grad(): # No gradient is required during validation
        print(f"Validating epoch {epoch+1}")
        val_loss = []
        for i, (inputs, y_true) in enumerate(tqdm(val_loader)):
            # Transfer data from cpu to gpu
            inputs = inputs.to(device)
            y_true = y_true.to(device)
            y_true = y_true.to(torch.int64)
            
            # Predict
            y_pred = model(inputs)

            # Calculate loss
            loss = loss_fn(y_pred, y_true)

            # Log stuff
            val_loss.append(loss)
        
        avg_val_loss = torch.stack(val_loss).mean().item()
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch+1} val loss: {avg_val_loss:.4f}")

        # LR adjustment with scheduler
        scheduler.step(avg_val_loss)

        # Save checkpoint if val_loss is the best we got
        best_val_loss = np.inf if epoch == 0 else min(val_losses[:-1])
        if avg_val_loss < best_val_loss:
            # Save whatever you want
            state = {
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'best_val_loss': best_val_loss,
            }
            
            print(f"Saving new best model..")
            torch.save(state, config['model_path'])

    # Check for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), config['best_model_path'])  # save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter == config['early_stop_patience']:
            print(f'Early stopping at epoch {epoch}')
            break
    wandb.log({
        'train_loss': avg_train_loss,
        'val_loss': avg_val_loss,
        'lr': current_lr,
    })

wandb.finish()
print('Finished Training')

# Evaluate model

In [ ]:
checkpoint = torch.load('')
import copy
best_model = copy.deepcopy(model)
best_model.load_state_dict(checkpoint) # Load weights

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True)
pred_val_list = []
for i in val_loader:
    # print(i[0].shape)
    # plt.imshow(i[0][0].permute(1, 2, 0))
    best_model.eval()
    with torch.no_grad():
        pred = best_model(i[0].to(device))
        pred = np.argmax(pred.to('cpu').numpy(), axis = 1).item()
        pred_val_list.append(pred)

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

print(classification_report(y_true=y_val, y_pred=pred_val_list))

cm = confusion_matrix(y_true=y_val, y_pred=pred_val_list)
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Run prediction for submission

In [ ]:
sample_sub = pd.read_csv('sample_submission.csv')

In [ ]:
sample_sub.head()

In [ ]:
test_data, _ = read_time_data(sample_sub, main_path = 'test/test/')

In [ ]:
test_data = UWBDataset(test_data, _)

In [ ]:
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, pin_memory=True)

In [ ]:
checkpoint = torch.load('results/efficientnetv2_s_lr0.01_range_time/best_model.pth')
import copy
test_model = copy.deepcopy(model)
test_model.load_state_dict(checkpoint) # Load weights

In [ ]:
pred_list = []
for i in test_loader:
    # print(i[0].shape)
    # plt.imshow(i[0][0].permute(1, 2, 0))
    test_model.eval()
    with torch.no_grad():
        pred = test_model(i[0].to(device))
        # print(pred)
        # break
        pred = np.argmax(pred.to('cpu').numpy(), axis = 1).item()
        pred_list.append(pred)

In [ ]:
sample_sub['class'] = pred_list

In [ ]:
sample_sub

In [ ]:
sample_sub.to_csv('results/efficientnetv2_s_lr0.01_range_time/kiddee3-range-time-3.csv', index = False)